In [ ]:
%pip install streamlit crewai-tools crewai google-search-results

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached crewai_tools-0.47.1-py3-none-any.whl (606 kB)


In [2]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests
from crewai import Agent, Task, Crew, Process
import os

In [47]:
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://www.tgsrtc.telangana.gov.in/tenders')

# Extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content
Tenders Home About Us About Home The Corporation TGSRTC Leadership IT Initiatives Events & Awards Re


In [48]:
from crewai_tools import FileWriterTool

class SafeFileWriterTool(FileWriterTool):
    def run(self, filename, content, directory='', overwrite=True):
        import os
        filepath = os.path.join(directory, filename)
        if os.path.exists(filepath) and not overwrite:
            return f"File '{filepath}' already exists. Overwrite not allowed."

        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                file.write(content)
            return f"Content written to '{filepath}'."
        except Exception as e:
            return f"An error occurred while writing to the file: {e}"


In [49]:
file_writer_tool = SafeFileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool.run(filename='ai.txt', content=text, directory='', overwrite=True)
print(result)


Content written to 'ai.txt'.


In [50]:
import os
from crewai_tools import TXTSearchTool

# os.environ['GROQ_API_KEY'] = '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'
# os.environ['GROQ_MODEL'] = 'groq/llama-3.1-8b-instant'
os.environ['OPENAI_API_KEY'] = 'sk-svcacct-A7-TCvp0gxsoY3tYlyYylhkSXfMhxlcf9Th10nZ97KN-3_Aeg09dfok1f-5rXp-HsT3BlbkFJhVGr7-SGP31NP9uACoI6s16yfT9q20sPBPqRxdb47C0S04KqdD-gkwAeZy19h8rAA'  # <-- this line fixes your KeyError

search_tool = TXTSearchTool(txt='ai.txt')


c:\Users\GUNDETI MANOJ\OneDrive\Desktop\Oreilly\myenv\Lib\site-packages\alembic\config.py:564: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
c:\Users\GUNDETI MANOJ\OneDrive\Desktop\Oreilly\myenv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


In [51]:
data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question.',
    backstory='You are a data expert to analyse the tenders names with last date for the rtc tenders and give me list of tenders which have last date as on or after 09-06-2025.',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the list of tenders with last date for the rtc tenders and give me list of tenders which have last date as on or after 09-06-2025.",
    agent=data_analyst,
    expected_output='give me list of tenders which have last date as on or after 09-06-2025.'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

# Agent: Educator
## Task: Understand the list of tenders with last date for the rtc tenders and give me list of tenders which have last date as on or after 09-06-2025.
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedTXTSearchToolSchema
search_query
  Input should be a valid string [type=string_type, input_value={'description': 'last dat...06-2025', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type.
 Tool Search a txt's content accepts these inputs: Tool Name: Search a txt's content
Tool Arguments: {'search_query': {'description': "Mandatory search query you want to use to search the txt's content", 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query the ai.txt txt's content.



# Agent: Educator
## Thought: I need to search the content for the tenders that have a last date on or after 09-06-2025.
## Using 

c:\Users\GUNDETI MANOJ\OneDrive\Desktop\Oreilly\myenv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x




# Agent: Educator
## Thought: Thought: I need to properly format the search query to find tenders with a last date on or after 09-06-2025.
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"last date on or after 09-06-2025\"}"
## Tool Output: 
Relevant Content:
CACs etc., of Secunderabad Region 09.06.2025 View Details 13(a) TENDER FORM for PARKING DRIVERS ,BUS WASHING SWEEPING , CACs TENDER FORM-2025 of Secunderabad Region View Details Quick Links Careers Site Map About Contact Us Information RTI Act, 2005 Road Safety Privacy Policies Feedback FAQs Reservations Bus Passes Follow us on @TGSRTCMDOFFICE Download our app Copyright © 2021 TGSRTC, All Rights Reserved.

Tenders Home About Us About Home The Corporation TGSRTC Leadership IT Initiatives Events & Awards Reservations Book e-Ticket Ticket Booking Agents KalaBhavan Booking Coach Types Bus on Contract Rates Marketing Schemes Special Bus Timings Tourism FAQs Logistics Book Your Cargo Rates and Tariff Contact

In [52]:
print(output.raw)

1. Notification for Security Guards for TGSRTC hospital Tarnaka etc., - Last Date: 09.06.2025
2. Notification for Sweeping, cleaning, and washing of bus stations and maintenance for Adilabad Region - Last Date: 12.06.2025

This includes the tenders that have last dates on or after 09-06-2025.


ABOVE PART IS OF WEBSCRAPING
BELOW IS BOTS INTEGRATION

In [8]:
# from summarize import SummarizationTool

summarization_tool = SummarizationTool()
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://www.uipath.com/blog/ai/agentic-ai-summit-highlights')

# Extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content
Agentic AI Summit recap: Embracing AI transformation enterprise-wide | UiPath Missed DevCon Online? 


In [59]:
from crewai_tools import FileWriterTool

class SafeFileWriterTool(FileWriterTool):
    def run(self, filename, content, directory='', overwrite=True):
        import os
        filepath = os.path.join(directory, filename)
        if os.path.exists(filepath) and not overwrite:
            return f"File '{filepath}' already exists. Overwrite not allowed."

        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                file.write(content)
            return f"Content written to '{filepath}'."
        except Exception as e:
            return f"An error occurred while writing to the file: {e}"


In [5]:
file_writer_tool = SafeFileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool.run(filename='blog.txt', content=text, directory='', overwrite=True)
print(result)


Content written to 'blog.txt'.


In [64]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import os
import requests

class SummarizationTool(BaseTool):  
    # Define required fields
    name: str = "SummarizationTool"  
    description: str = "A tool to summarize text using a large language model."
    api_key: str = Field(default_factory=lambda: os.getenv("GROQ_API_KEY"))
    model: str = "groq/llama-3.3-70b-versatile"
    base_url: str = "https://api.groq.com/openai/v1/chat/completions "
    
    def _run(self, text: str, max_words: int = 50) -> str:
        """
        Summarize the given text using the summarization API.
        
        This method implements the abstract '_run' method.
        
        Args:
            text (str): Text to be summarized
            max_words (int, optional): Maximum number of words in summary
        
        Returns:
            str: Summarized text
        """
        try:
            # Validate input
            if not text or not isinstance(text, str):
                return "Error: Invalid input. Please provide a non-empty string."
            
            # Construct prompt for summarization
            prompt = f"""
            Summarize the following text concisely, capturing the most important points:
            
            TEXT:
            {text}
            
            Guidelines:
            - Preserve the main ideas and key insights
            - Be extremely concise
            - Limit the summary to approximately {max_words} words
            - Focus on the core message of the text
            """
            
            # Prepare API request payload
            payload = {
                "model": self.model,
                "messages": [
                    {
                        "role": "system", 
                        "content": "You are a helpful assistant skilled in creating concise summaries."
                    },
                    {
                        "role": "user", 
                        "content": prompt
                    }
                ],
                "temperature": 0.3,
                "max_tokens": 150  # Adjust based on max_words
            }
            
            # Make API call
            response = requests.post(
                self.base_url,
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json=payload
            )
            
            # Check response
            response.raise_for_status()
            
            # Extract summary
            result = response.json()
            summary = result['choices'][0]['message']['content'].strip()
            
            return summary
        
        except requests.RequestException as e:
            return f"API Request Error: {str(e)}"
        except Exception as e:
            return f"Summarization error: {str(e)}"
        

In [48]:
from crewai.tools import tool

@tool("Local File Reader")
def read_local_file() -> str:
    """Reads the content of a local file named blog.txt."""
    try:
        with open("blog.txt", "r", encoding="utf-8") as f:
            content = f.read()
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"


In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Set environment variables
os.environ['GROQ_API_KEY'] = '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'
os.environ['GROQ_MODEL'] = 'groq/llama3-70b-8192'  # correct model name
# os.environ['OPENAI_API_KEY'] = 'sk-svcacct-A7-TCvp0gxsoY3tYlyYylhkSXfMhxlcf9Th10nZ97KN-3_Aeg09dfok1f-5rXp-HsT3BlbkFJhVGr7-SGP31NP9uACoI6s16yfT9q20sPBPqRxdb47C0S04KqdD-gkwAeZy19h8rAA'  # for any openai integration if needed


import os
import requests
from pydantic import BaseModel, Field
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from crewai.tools import tool

# --------------------------------------------------
# File Reader Tool
# --------------------------------------------------
@tool("Local File Reader")
def read_local_file() -> str:
    """Reads the content of a local file named blog.txt."""
    try:
        with open("blog.txt", "r", encoding="utf-8") as f:
            content = f.read()
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"

# --------------------------------------------------
# Summarization Tool using Groq API
# --------------------------------------------------
# class SummarizationTool(BaseTool):
#     name: str = "SummarizationTool"
#     description: str = "A tool to summarize text using a large language model."
#     api_key: str = Field(default_factory=lambda: os.getenv("GROQ_API_KEY"))
#     model: str = "groq/llama-3.3-70b-versatile"
#     base_url: str = "https://api.groq.com/openai/v1/chat/completions"

#     def _run(self, text: str, max_words: int = 500) -> str:
#         try:
#             if not text or not isinstance(text, str):
#                 return "Error: Invalid input. Please provide a non-empty string."

#             prompt = f"""
#             Summarize the following text concisely, capturing the most important points:

#             TEXT:
#             {text}

#             Guidelines:
#             - Preserve the main ideas and key insights
#             - Be extremely concise
#             - Limit the summary to approximately {max_words} words
#             - Focus on the core message of the text
#             """

#             payload = {
#                 "model": self.model,
#                 "messages": [
#                     {"role": "system", "content": "You are a helpful assistant skilled in creating concise summaries."},
#                     {"role": "user", "content": prompt}
#                 ],
#                 "temperature": 0.3,
#                 "max_tokens": 1500  # roughly allows ~500 words
#             }

#             response = requests.post(
#                 self.base_url,
#                 headers={
#                     "Authorization": f"Bearer {self.api_key}",
#                     "Content-Type": "application/json"
#                 },
#                 json=payload
#             )

#             response.raise_for_status()
#             result = response.json()
#             summary = result['choices'][0]['message']['content'].strip()

#             return summary

#         except requests.RequestException as e:
#             return f"API Request Error: {str(e)}"
#         except Exception as e:
#             return f"Summarization error: {str(e)}"

# --------------------------------------------------
# Instantiate Tools
# --------------------------------------------------
file_reader_tool = read_local_file
summarization_tool = SummarizationTool()

# --------------------------------------------------
# Agents
# --------------------------------------------------
file_reader_agent = Agent(
    role="File Reader",
    goal="Read content from blog.txt file",
    backstory="You are an expert at reading local files and extracting the full content accurately for further processing.",
    tools=[file_reader_tool],
    verbose=True
)

summarizer_agent = Agent(
    role="Summarizer",
    goal="Summarize the content of the blog into maximum 500 words.",
    backstory="You specialize in summarizing long articles into clear, insightful summaries that capture the core message.",
    tools=[summarization_tool],
    verbose=True
)

# --------------------------------------------------
# Tasks
# --------------------------------------------------
read_file_task = Task(
    description="Read the complete content from the local file named 'blog.txt' and provide its full text content.",
    expected_output="The entire content of the blog.txt file.",
    agent=file_reader_agent
)

summarize_task = Task(
    description="Using the extracted content, generate a concise summary of the blog within 500 words.",
    expected_output="A clean, well-written summary of the blog.txt contents, strictly under 500 words.",
    agent=summarizer_agent
)

# --------------------------------------------------
# Crew
# --------------------------------------------------
crew = Crew(
    agents=[file_reader_agent, summarizer_agent],
    tasks=[read_file_task, summarize_task],
    process=Process.sequential
)

# --------------------------------------------------
# Execution
# --------------------------------------------------
output = crew.kickoff()
print("Final Output:")
print(output.raw)

# Agent: File Reader
## Task: Read the complete content from the local file named 'blog.txt' and provide its full text content.


# Agent: File Reader
## Thought: Thought: I need to read the content from the blog.txt file to provide the complete text.
## Using tool: Local File Reader
## Tool Input: 
"{}"
## Tool Output: 
Agentic AI Summit recap: Embracing AI transformation enterprise-wide | UiPath Missed DevCon Online? Catch up now Explore Explore Explore Agentic AI See how AI and automation combine to create powerful autonomous agents. What is RPA Discover the essentials of RPA, the automation software to end repetitive tasks. UiPath.ai Go behind the scenes into our technology, meet our world-class scientists, and learn about our cutting-edge AI research. Agentic Agentic AI Agentic automation Agentic testing AI agents AI and RPA Generative AI Specialized AI Enterprise AI AI Assistant Robotic What is RPA SAP automation AI automation Web automation Desktop automation Email automation No

In [62]:
file_writer_tool = SafeFileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool.run(filename='blog_summary.txt', content=output.raw, directory='', overwrite=True)
print(result)


Content written to 'blog_summary.txt'.
